In [2]:
import pandas as pd
from DataPreprocessor import DataPreprocessor
import pymysql

In [3]:
# !pip install sqlalchemy
# !pip install python-dotenv


In [4]:
start_date = '2024-09-01'
end_date = '2024-11-20'
floor = '5'
zone_side = '북동측'

In [5]:
# 동적 쿼리
query = f"""
WITH energy_data AS (
    SELECT TO_CHAR(DATE_TRUNC('hour', zpd.recorded_at) +
                    INTERVAL '15 min' * (EXTRACT(MINUTE FROM zpd.recorded_at)::INTEGER / 15),
                    'YYYY-MM-DD HH24:MI') AS timerange,
            zpd.zone_prepared_data_plug_usage AS plug_use,
            zpd.zone_prepared_data_lighting_usage AS light_use,
            zpd.zone_prepared_data_ventilation_usage AS ventilation_use,
            zpd.zone_prepared_data_heating_usage AS heating_use,
            zpd.zone_prepared_data_cooling_usage AS cooling_use
    FROM v2.zone_prepared_data zpd
            JOIN v2.zone z ON zpd.zone_id = z.zone_id
    WHERE z.zone_name LIKE '%{floor}F {zone_side}%'  -- floor와 zone_side를 동적으로 설정
        AND zpd.recorded_at >= '{start_date}'
        AND zpd.recorded_at < '{end_date}'
),
environment_data AS (
    SELECT TO_CHAR(DATE_TRUNC('hour', ed.recorded_at) +
                    INTERVAL '15 min' * (EXTRACT(MINUTE FROM ed.recorded_at)::INTEGER / 15),
                    'YYYY-MM-DD HH24:MI') AS timerange,
            AVG(CASE WHEN mp.measurement_point_subtype = 'temperature' THEN ed.environment_data_value END) AS temperature,
            AVG(CASE WHEN mp.measurement_point_subtype = 'humidity' THEN ed.environment_data_value END) AS humidity,
            AVG(CASE WHEN mp.measurement_point_subtype = 'co2' THEN ed.environment_data_value END) AS co2,
            AVG(CASE WHEN mp.measurement_point_subtype = 'pm10' THEN ed.environment_data_value END) AS pm10,
            AVG(CASE WHEN mp.measurement_point_subtype = 'pm25' THEN ed.environment_data_value END) AS pm25,
            AVG(CASE WHEN mp.measurement_point_subtype = 'tvoc' THEN ed.environment_data_value END) AS tvoc,
            AVG(CASE WHEN mp.measurement_point_subtype = 'illuminance' THEN ed.environment_data_value END) AS illuminance
    FROM v2.environment_data ed
            JOIN v2.measurement_point mp ON ed.measurement_point_id = mp.measurement_point_id
            JOIN v2.zone z ON mp.environment_sensor_device_zone_id = z.zone_id
    WHERE z.zone_name LIKE '%{floor}F {zone_side}%'  -- floor와 zone_side를 동적으로 설정
        AND ed.recorded_at >= '{start_date}'
        AND ed.recorded_at < '{end_date}'
    GROUP BY timerange
)
SELECT ed.*,
        env.temperature AS temperature_15min_avg,
        env.humidity AS humidity_15min_avg,
        env.co2 AS co2_15min_avg,
        env.pm10 AS pm10_15min_avg,
        env.pm25 AS pm25_15min_avg,
        env.tvoc AS tvoc_15min_avg,
        env.illuminance AS illuminance_15min_avg
FROM energy_data ed
        JOIN environment_data env ON ed.timerange = env.timerange;
"""

In [ ]:
procs = DataPreprocessor(column_name="plug_use")
procs.load_data(query_template=query)

SSH 터널 활성화됨 - 로컬 포트: 55028


In [ ]:
raw_data = procs.df.copy()

In [ ]:
raw_data.head()

,plug_use,light_use,ventilation_use,heating_use,cooling_use,temperature_15min_avg,humidity_15min_avg,co2_15min_avg,pm10_15min_avg,pm25_15min_avg,tvoc_15min_avg,illuminance_15min_avg
timerange,,,,,,,,,,,,
2024-09-01 09:00:00,0.128581,0.0,0.006304,0.001804,0.042059,27.088889,66.964445,426.977778,11.955556,11.511111,583.866667,2.488889
2024-09-01 09:15:00,0.127297,0.0,0.005500,0.001815,0.042465,27.017778,67.095555,426.733333,11.711111,11.311111,574.488889,2.400000
2024-09-01 09:30:00,0.129991,0.0,0.005750,0.001813,0.042222,26.964445,67.231111,425.800000,11.288889,10.911111,564.288889,2.488889
2024-09-01 09:45:00,0.097439,0.0,0.006488,0.001814,0.042451,26.922223,67.346666,425.355556,10.733333,10.555556,551.355556,2.444444
2024-09-01 10:00:00,0.128581,0.0,0.005517,0.001813,0.042232,26.857778,67.477776,425.044444,10.577778,10.400000,537.666667,2.577778


In [ ]:
raw_data.shape

(7546, 12)

In [ ]:
raw_data.columns

Index(['plug_use', 'light_use', 'ventilation_use', 'heating_use',
       'cooling_use', 'temperature_15min_avg', 'humidity_15min_avg',
       'co2_15min_avg', 'pm10_15min_avg', 'pm25_15min_avg', 'tvoc_15min_avg',
       'illuminance_15min_avg'],
      dtype='object')

In [ ]:
RAW_DATA_PATH = f'C:/Users/User/Desktop/바탕화면/승건/데사팀/lskeantech_code/lskeantec/삼성물산_재실_예측/data/{floor}F_{zone_side}.pkl'

In [ ]:
# 데이터 저장
raw_data.to_pickle(RAW_DATA_PATH)
print(f"Raw data saved to {RAW_DATA_PATH}")

Raw data saved to C:/Users/User/Desktop/바탕화면/승건/데사팀/lskeantech_code/lskeantec/삼성물산_재실_예측/data/5F_남서측.csv
